# TechXchange Japan 2024: さわってみようベクトル・データベース watsonx.dataでRAG体験


生成AIの回答精度を向上させるために、自社内のデータを活用してみましょう！<br>
ベクトル・データベース + 大規模言語モデル（LLM）で構成されるRAGのアーキテクチャーを使えば、自社内の情報で生成AIチャットボットが作成できます。<br>
当ハンズオンでは「TechXchange Japan 2024」に関するデータを使ってRAGを構成し、「TechXchange Japan 2024」に関することを教えてくれるチャットボットを作成します。<br>
(以下ベクトル・データベースはベクトルDBと表記します。)<br>


具体的には大規模言語モデル（LLM）を使用したアプリケーション開発のためのオープンソース・オーケストレーション・フレームワーク[LangChain](https://python.langchain.com/docs/introduction/)を使って、wastosonx.dataのベクトルDB **Milvus**に「TechXchange Japan 2024」に関するデータをロードし、watson.aiで提供されているLLMを使用してRAGを構成し、「TechXchange Japan 2024」のことを回答してくれるChatbotを作ってみます。

ハンズオンは以下の順序で実行します:

1. **Excelをベクトル化してベクトルDB Milvusに入れよう！** [**当notebook**]
2. ベクトルDB Milvusに入ったデータで類似検索してみよう!
3. ベクトルDB Milvusとwatsonx.ai LLMでRAGを構成して、質問をしてみよう!
4. ベクトルDB Milvusとwatsonx.ai LLMでRAGを構成して、チャットアプリを作成してみよう!

## 1. Excelをベクトル化してベクトルDB Milvusに入れよう！

### 1. 必要なライブラリーのインストール

In [ ]:
!pip install -Uq 'ibm-watsonx-ai>=1.1.15'
!pip install -Uq 'langchain>=0.3.3'
!pip install -Uq 'langchain-ibm>=0.3.1'
!pip install -Uq 'langchain-huggingface>=0.1.0'
!pip install -Uq 'langchain-milvus>=0.1.6'
!pip install -Uq 'langchain-community>=0.3.2'
!pip install -Uq 'pymilvus>=2.4.8'

In [ ]:
!pip check

`langchain-chroma`と`langchain-elasticsearch`は今回使用していないので問題ないです。<br>

**インストール終了後、一旦カーネルを再起動してください** <br>

**手順:**
- 上部のメニュー「Karnel」から「Restart Karnel and Clear Outputs of All Cells...」をクリック
- 「Restart Kernel?」 のダイアログが表示されるので、「Restart」をクリック

### 2. apikeyの設定 

- 事前に取得したapikeyを<api_key>　に入れる
    - 例:  `apikey="YyyyyyyyXxxxxxxxxxxxxZzzzzzzzzzzzzz"`


In [1]:
apikey="<api_key>"

### 3. Milvus接続情報の設定

- watsonx.dataの画面を開く
- ナビゲーションメニューから「インフラストラクチャー・マネージャー」を選択
- サービス「Milvus」をクリック
- タイプの下の「接続の詳細を見る」をクリック
- GRPC ホストの値を<milvus GRPC ホスト>　に入れる
    - 例:  `milvus_host="xxxxxxxxxxx-xxxxxxxxxxx-xxxxxxxx.xxxxxxxx.lakehouse.appdomain.cloud"`
- GRPC ポートの値を<milvus GRPC ポート>　に入れる
    - 例:  `milvus_port="9999"`

In [ ]:
milvus_host="<milvus GRPC ホスト>"
milvus_port="<milvus GRPC ポート>"

In [ ]:


my_connection_args ={
 'uri': f'https://{milvus_host}:{milvus_port}', 
 'token': f'ibmlhapikey:{apikey}'
}

### 4. Excelデータの取得
TechXchange Confrence Japan 2024の情報の入ったExcelを取得します。

In [ ]:
!wget https://github.com/IBM/japan-technology/raw/refs/heads/main/techxchange/2024-watsonx-handson-1/data/TechXchangeJapan2024.xlsx -O TechXchangeJapan2024.xlsx

In [ ]:
# ファイルの確認
!ls -la

### 5. 必要ライブラリーのImport

In [ ]:
import pandas as pd
from langchain.schema.document import Document
import json
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_milvus import Milvus
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

### 6. Excelファイルの内容を pandas Daraframeに読み込む

In [ ]:
path="./"
filename='TechXchangeJapan2024.xlsx'
excel_file = path+filename

df_list = []

# 全てのシートを読み込み、リストdf_listに格納
for sheet_name in pd.ExcelFile(excel_file).sheet_names:
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    df_list.append(df)
    display(df.head()) #各シート最初の5　行　表示

### 7. 行をJSON化し、metadataとして`Category`,`ID`を抜き出す

ベクトルDBにデータを入れる際、どの単位でどのようにベクトル化するかというのは、のちのちの類似検索の結果に関わってきますので重要です。

本日はやりませんが、PDFならページ単位に文字を抜き出して、1ページ分を1ベクトルににするとか、さらに細かく切っておおよそ1000文字単位でうまく文章の切れ目で切って1ベクトルにするとか、いろいろ考えられます。

今回はExcelファイルなので、シート単位で1シート1ベクトルとか、一行1ベクトルとかが考えられます。<br>
今回は一行1ベクトルにしてみます。<br>
さらに一行のベクトル化する元の文字列ですが、列名をいれたJSON形式の文字列にしてみます。<br>

Milvusにはベクトルデータの他に、Keyとなる値を列として持つことができます。RDBのように列で検索も可能です。

[LangChain](https://python.langchain.com/docs/introduction/)を使って、ベクトルDBにインサートする際、列データはDocumentオブジェクトのmetadataとしてJSONで指定します。
こちらのmetadataのJSON文字列もここで作成します。

In [ ]:
json_doc_list=[]
json_meta_list=[]

for df in df_list:
    # 各シートのデータフレームに対する処理
    
    # 行をJSONフォーマットに変換
    json_doc_string = json.loads(df.to_json(orient='records', force_ascii=False))

    # metaデータとして'Category','ID'を抜き出し, JSONに変換
    json_meta_string =  json.loads(df[['Category','ID']].to_json(orient='records', force_ascii=False), parse_int=str)

    #　各シートのJSON　Listを1つのListに結合
    json_doc_list.extend(json_doc_string)
    json_meta_list.extend( json_meta_string)

#中身確認
print(json_doc_list[0])
print(json_meta_list[0])

### 8. 1行の情報をlangchainのDocumentにし、Listを作成
- page_contentはjson_doc_listの一行分のjson
    - ベクトル化されるデータ
- metadataはjson_meta_listのの一行分のjson
    - ベクトルDBに列の項目として入るデータ

In [ ]:
# DocumentのListをjson_doc_listとjson_meta_listから作成
# page_contentはjson_doc_listの一行分のjson
# metadataはjson_meta_listのの一行分のjson
docs = [Document(page_content=json.dumps(doc_str, ensure_ascii=False), metadata=meta_str) 
        for doc_str, meta_str in zip(json_doc_list, json_meta_list)]

#中身確認
print(docs[0])
print(docs[1])

### 9. Embeddingモデルを作成
ここでは`intfloat/multilingual-e5-large`を使います<br>
https://huggingface.co/intfloat/multilingual-e5-large

In [ ]:
from tqdm.autonotebook import tqdm
embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

#### 10.  ベクトルDB Milvusにデータの挿入

In [ ]:
# パラメータの設定 
# metric_type https://milvus.io/docs/ja/metric.md?tab=floating 参照
# index_type https://milvus.io/docs/index.md?tab=floating　参照
index_params = {
    "metric_type": "COSINE", #コサイン類似度
    "index_type": "HNSW", 
    "params": { "M": 16,"efConstruction": 200,"efSearch": 16 }
}

# techxchange_line_data に　データの挿入
vector_db = Milvus.from_documents(
    docs,
    embeddings,
    connection_args=my_connection_args,
    index_params = index_params,
    drop_old=True, #追加の場合はここをFalseに
    collection_name = 'techxchange_line_data'
)

(既にデータの入っているMilvusのCollectionに接続する場合は以下を使用してください)

In [ ]:
# 既存データを使う場合はこちらを実行
from langchain_milvus import Milvus

vector_db = Milvus(
    embeddings,
    connection_args=my_connection_args,
    collection_name = 'techxchange_line_data'
)

### 11.  挿入データの確認

Milvus DBにロードした内容をDataFrameにダンプして表示させます。

In [ ]:
from pymilvus import MilvusClient
import pandas as pd

pk_list=vector_db.get_pks(expr="pk > 0")

client = MilvusClient(uri=my_connection_args['uri'], token=my_connection_args['token'])

res = client.get(
    collection_name="techxchange_line_data",
    ids=pk_list
)

for i, milvus_rec in enumerate(res):
    vecter_data = milvus_rec['vector']
    res[i]['vector']= "["+", ".join(map(str, vecter_data))+"]"
    res[i]['pk']= str(milvus_rec['pk'])


df_s = pd.DataFrame.from_dict(res).reindex(columns=['ID', 'Category', 'text', 'pk', 'vector'])
df_s

これで「1. Excelをベクトル化してベクトル・データベース Milvusに入れよう！」は完了です。<br>
次の「2. ベクトル・データベース Milvusに入ったデータで類似検索してみよう!」に進んでください。